In [13]:
# General
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools as it

# CV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score,train_test_split,RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import GridSearchCV, ParameterGrid
import time as time

# Models
from sklearn.linear_model import LinearRegression
from patsy import dmatrix
# from pyearth import Earth
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor

In [2]:
data = pd.read_csv('project_data.csv')
data.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505


In [3]:
data.describe()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
count,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,...,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000,39644.000000
mean,354.530471,10.398749,546.514731,0.530545,0.970201,0.672790,10.883690,3.293638,4.544143,1.249874,...,0.095446,0.756728,-0.259524,-0.521944,-0.107500,0.282353,0.071425,0.341843,0.156064,3395.380184
std,214.163767,2.114037,471.107508,0.137111,0.170017,0.154039,11.332017,3.855141,8.309434,4.107855,...,0.071315,0.247786,0.127726,0.290290,0.095373,0.324247,0.265450,0.188791,0.226294,11626.950749
min,8.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,1.000000
25%,164.000000,9.000000,246.000000,0.470858,1.000000,0.625720,4.000000,1.000000,1.000000,0.000000,...,0.050000,0.600000,-0.328383,-0.700000,-0.125000,0.000000,0.000000,0.166667,0.000000,946.000000
50%,339.000000,10.000000,409.000000,0.539216,1.000000,0.690476,8.000000,3.000000,1.000000,0.000000,...,0.100000,0.800000,-0.253333,-0.500000,-0.100000,0.150000,0.000000,0.500000,0.000000,1400.000000
75%,542.000000,12.000000,716.000000,0.608696,1.000000,0.754620,14.000000,4.000000,4.000000,1.000000,...,0.100000,1.000000,-0.186905,-0.300000,-0.050000,0.500000,0.150000,0.500000,0.250000,2800.000000
max,731.000000,23.000000,8474.000000,1.000000,1.000000,1.000000,304.000000,116.000000,128.000000,91.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.500000,1.000000,843300.000000


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['shares', 'url'], axis = 1),
    data['shares'],
    test_size = 0.2,
    random_state = 42)

print(X_train.shape)
print(X_test.shape)

(31715, 59)
(7929, 59)


In [5]:
train = pd.concat([X_train, y_train], axis = 1)

# VIF

In [21]:
train_alpha = train.reindex(sorted(train.columns), axis = 1)
train_alpha.head()

,abs_title_sentiment_polarity,abs_title_subjectivity,average_token_length,avg_negative_polarity,avg_positive_polarity,data_channel_is_bus,data_channel_is_entertainment,data_channel_is_lifestyle,data_channel_is_socmed,data_channel_is_tech,...,timedelta,title_sentiment_polarity,title_subjectivity,weekday_is_friday,weekday_is_monday,weekday_is_saturday,weekday_is_sunday,weekday_is_thursday,weekday_is_tuesday,weekday_is_wednesday
19837,0.333333,0.041667,5.090764,-0.259184,0.275971,1.0,0.0,0.0,0.0,0.0,...,338.0,0.333333,0.458333,0.0,0.0,0.0,0.0,0.0,1.0,0.0
904,0.000000,0.500000,4.958435,-0.133333,0.295343,0.0,0.0,0.0,0.0,0.0,...,715.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0
23876,0.000000,0.500000,4.942029,-0.282639,0.428715,0.0,0.0,0.0,1.0,0.0,...,262.0,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0
22966,0.200000,0.050000,4.520737,-0.098214,0.325667,0.0,0.0,0.0,1.0,0.0,...,279.0,0.200000,0.550000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4705,0.000000,0.500000,4.479675,-0.550000,0.444524,0.0,0.0,0.0,0.0,1.0,...,647.0,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [24]:
dummies = [col for col in train_alpha.columns if 'is' in col]
dummies

['data_channel_is_bus',
 'data_channel_is_entertainment',
 'data_channel_is_lifestyle',
 'data_channel_is_socmed',
 'data_channel_is_tech',
 'data_channel_is_world',
 'is_weekend',
 'weekday_is_friday',
 'weekday_is_monday',
 'weekday_is_saturday',
 'weekday_is_sunday',
 'weekday_is_thursday',
 'weekday_is_tuesday',
 'weekday_is_wednesday']

In [27]:
dropped = ['data_channel_is_bus', 'is_weekend', 'weekday_is_friday']
train_dropped = train.drop(dropped, axis = 1)
train_dropped.head()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
19837,338.0,15.0,628.0,0.509709,1.0,0.645084,14.0,1.0,1.0,0.0,...,0.033333,0.8,-0.259184,-0.500000,-0.100000,0.458333,0.333333,0.041667,0.333333,16100
904,715.0,11.0,409.0,0.519608,1.0,0.657480,2.0,0.0,1.0,0.0,...,0.136364,0.5,-0.133333,-0.166667,-0.100000,0.000000,0.000000,0.500000,0.000000,508
23876,262.0,7.0,345.0,0.609329,1.0,0.783920,5.0,2.0,1.0,0.0,...,0.062500,0.6,-0.282639,-0.800000,-0.050000,0.000000,0.000000,0.500000,0.000000,1300
22966,279.0,11.0,217.0,0.580189,1.0,0.775000,8.0,2.0,3.0,0.0,...,0.033333,0.5,-0.098214,-0.125000,-0.071429,0.550000,0.200000,0.050000,0.200000,3100
4705,647.0,13.0,123.0,0.739837,1.0,0.887324,3.0,0.0,1.0,0.0,...,0.200000,1.0,-0.550000,-1.000000,-0.100000,0.000000,0.000000,0.500000,0.000000,6900


In [30]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


X = train_dropped

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
def vif(X):
    X = add_constant(X)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X.columns

    for i in range(len(X.columns)):
        vif_data.loc[i,'VIF'] = variance_inflation_factor(X.values, i)

    return vif_data

In [31]:
vif(X).sort_values(by = 'VIF', ascending = False)

,feature,VIF
0,const,39016.212503
39,lda_03,3370.451782
40,lda_04,3266.268768
38,lda_02,3097.387119
36,lda_00,2678.574125
37,lda_01,1889.767944
45,rate_positive_words,576.303549
5,n_non_stop_words,478.287376
46,rate_negative_words,390.994074
4,n_unique_tokens,20.818133


In [37]:
#Remove collinear predictors one at a time
remove = ['lda_03']
pred_filter = [x for x in train_dropped.columns if x not in remove]
X = train[pred_filter]

#Recompute VIF
vif(X).sort_values(by = 'VIF', ascending = False)

,feature,VIF
44,rate_positive_words,473.125533
5,n_non_stop_words,397.910468
45,rate_negative_words,321.620323
0,const,226.117708
4,n_unique_tokens,20.817773
6,n_non_stop_unique_tokens,18.958656
29,self_reference_avg_sharess,16.151240
11,average_token_length,12.767703
19,kw_max_min,11.361359
20,kw_avg_min,10.967783


In [38]:
#Remove collinear predictors one at a time
remove = ['lda_03', 'rate_positive_words']
pred_filter = [x for x in train_dropped.columns if x not in remove]
X = train[pred_filter]

#Recompute VIF
vif(X).sort_values(by = 'VIF', ascending = False)

,feature,VIF
0,const,226.075888
4,n_unique_tokens,20.810574
5,n_non_stop_words,19.438916
6,n_non_stop_unique_tokens,18.958158
29,self_reference_avg_sharess,16.150345
11,average_token_length,12.767702
19,kw_max_min,11.361359
20,kw_avg_min,10.967782
26,kw_avg_avg,10.738339
44,rate_negative_words,9.261724


In [39]:
#Remove collinear predictors one at a time
remove = ['lda_03', 'rate_positive_words', 'n_unique_tokens', 'n_non_stop_words']
pred_filter = [x for x in train_dropped.columns if x not in remove]
X = train[pred_filter]

#Recompute VIF
vif(X).sort_values(by = 'VIF', ascending = False)

,feature,VIF
0,const,224.449017
27,self_reference_avg_sharess,16.143361
17,kw_max_min,11.357841
18,kw_avg_min,10.963160
24,kw_avg_avg,10.721408
42,rate_negative_words,8.823429
26,self_reference_max_shares,8.140720
39,global_sentiment_polarity,7.798985
46,avg_negative_polarity,7.680582
23,kw_max_avg,7.482828


In [40]:
#Remove collinear predictors one at a time
remove = ['lda_03', 'rate_positive_words', 'n_unique_tokens', 'n_non_stop_words', 'self_reference_avg_sharess']
pred_filter = [x for x in train_dropped.columns if x not in remove]
X = train[pred_filter]

#Recompute VIF
vif(X).sort_values(by = 'VIF', ascending = False)

,feature,VIF
0,const,224.390301
17,kw_max_min,11.346233
18,kw_avg_min,10.958507
24,kw_avg_avg,10.714795
41,rate_negative_words,8.823420
38,global_sentiment_polarity,7.798910
45,avg_negative_polarity,7.680095
23,kw_max_avg,7.471457
40,global_rate_negative_words,6.506597
42,avg_positive_polarity,6.086198


In [41]:
#Remove collinear predictors one at a time
remove = ['lda_03', 'rate_positive_words', 'n_unique_tokens', 'n_non_stop_words', 
          'self_reference_avg_sharess', 'kw_max_min']
pred_filter = [x for x in train_dropped.columns if x not in remove]
X = train[pred_filter]

#Recompute VIF
vif(X).sort_values(by = 'VIF', ascending = False)

,feature,VIF
0,const,224.357439
23,kw_avg_avg,9.715426
40,rate_negative_words,8.822515
37,global_sentiment_polarity,7.798190
44,avg_negative_polarity,7.680079
39,global_rate_negative_words,6.506509
22,kw_max_avg,6.417215
41,avg_positive_polarity,6.085946
34,lda_02,5.264899
45,min_negative_polarity,5.240633


In [43]:
col_dropped = dropped + remove
col_dropped

['data_channel_is_bus',
 'is_weekend',
 'weekday_is_friday',
 'lda_03',
 'rate_positive_words',
 'n_unique_tokens',
 'n_non_stop_words',
 'self_reference_avg_sharess',
 'kw_max_min']